# Load COOP Test Data
## load example filings for the ledger (not complete filings)
downstream integrations (like receipts) wont actually work

### Setup Environment

In [ ]:
# using TEST database
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

In [ ]:
%run /workspaces/lear/tests/data/common/legal_api_utils_test_data.ipynb

In [ ]:
import copy
import re
import datedelta
from legal_api.core import Filing as CoreFiling
from legal_api.models import Comment, Filing, RegistrationBootstrap
from legal_api.utils.datetime import date, datetime, timezone

### Setup the Business Info

In [ ]:
identifier = 'CP0000840'
legal_type = identifier[:re.search(r"\d", identifier).start()]
identifier, legal_type

In [ ]:
if business := legal_api.models.Business.find_by_identifier(identifier):
        print(f'{identifier} already exisit in LEAR, DO NOT ATTENP TO LOAD IT AGAIN')
assert not business, "Business already exists, if you want to build it, set build to True and run the next cell."

In [ ]:
create_business = True
if create_business:
    current_time = datetime.utcnow()
    business = Business(legal_name=f'legal_name-{identifier}',
                        founding_date=(current_time - datedelta.datedelta(years=4)),
                        last_ar_date=current_time,
                        last_ar_year=current_time.year,
#                         last_coa_date=current_time,
#                         last_cod_date=current_time,
                        last_ledger_timestamp=current_time,
                        # dissolution_date=EPOCH_DATETIME,
                        identifier=identifier,
#                         tax_id='BN123456789',
                        fiscal_year_end_date=current_time,
                        legal_type=legal_type)
    business.save()
business.id, business.legal_name, business.founding_date

In [ ]:
bootstrap = RegistrationBootstrap()
bootstrap.identifier = 'T22121212'
bootstrap.account = 446
bootstrap.save()

### Create Incorporation Application

In [ ]:
filing_name = 'incorporationApplication'
filing_date = datetime(2017, 10, 1, 0, 2, 26, 768706, tzinfo=timezone.utc)
# filing_date = business.founding_date
filing_month = 0
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing'].pop('business')
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = INCORPORATION
filing_json
# filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)
filing = factory_pending_IA_filing(bootstrap, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data
filing.save()

In [ ]:
filing.id

In [ ]:
filing_name = 'incorporationApplication'
filing_date = business.founding_date
filing_month = 0
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing'].pop('business')
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = INCORPORATION
filing_json

In [ ]:
ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Incorporation Application'

### Create Alteration

In [ ]:
filing_name = 'alteration'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = ALTERATION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
    'fromLegalType': 'BC',
    'toLegalType': 'BEN'
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing.court_order_file_number = ALTERATION['courtOrder']['fileNumber']
filing.court_order_date = ALTERATION['courtOrder']['orderDate']
filing.court_order_effect_of_order = ALTERATION['courtOrder']['effectOfOrder']
filing.order_details = ALTERATION['courtOrder']['orderDetails']

filing._meta_data = meta_data
filing.save()

In [ ]:
ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Alteration'
# assert ledger_filing['data'] == meta_data 

### Create Change of Address

In [ ]:
filing_name = 'changeOfAddress'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = CHANGE_OF_ADDRESS
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Address Change'
assert ledger_filing['data'] == meta_data

### Create Change of Directors

In [ ]:
filing_name = 'changeOfDirectors'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = CHANGE_OF_DIRECTORS
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Director Change'
assert ledger_filing['data'] == meta_data

### Create Annual Report

In [ ]:
filing_name = 'annualReport'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = ANNUAL_REPORT
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

iso_date = filing_date.date().isoformat()
filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
        'annualReportDate': iso_date,
        'annualGeneralMeetingDate': iso_date
}}
meta_data = {**{'applicationDate': iso_date}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == f'Annual Report ({date.fromisoformat(iso_date).year})'
assert ledger_filing['data'] == meta_data

### Create Conversion

In [ ]:
filing_name = 'conversion'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = CONVERSION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Conversion'
assert ledger_filing['data'] == meta_data

### Create Special Resolution

In [ ]:
filing_name = 'specialResolution'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = SPECIAL_RESOLUTION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Special Resolution'
assert ledger_filing['data'] == meta_data

### Create Dissolution

In [ ]:
filing_name = 'dissolution'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = SPECIAL_RESOLUTION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Dissolution'
assert ledger_filing['data'] == meta_data

### Create Transition

In [ ]:
filing_name = 'transition'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = SPECIAL_RESOLUTION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Transition Application'
assert ledger_filing['data'] == meta_data

### Create Court Order

In [ ]:
filing_name = 'courtOrder'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = COURT_ORDER
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

iso_date = filing_date.date().isoformat()
filing_meta = {
    'legalFilings':[filing_name,],
}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}
filing._meta_data = meta_data

filing.court_order_file_number = 'ABC123'
filing.court_order_date = iso_date
filing.court_order_effect_of_order = 'planOfArrangement'
filing.order_details = 'my order details'

filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Court Order'
# assert ledger_filing['data'] == meta_data

### Create Registrars Notation

In [ ]:
filing_name = 'registrarsNotation'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = REGISTRARS_NOTATION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

iso_date = filing_date.date().isoformat()
filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': iso_date}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == "Registrar's Notation"
assert ledger_filing['data'] == meta_data

In [ ]:
REGISTRARS_ORDER

### Create Registrars Order

In [ ]:
filing_name = 'registrarsOrder'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = REGISTRARS_ORDER
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

iso_date = filing_date.date().isoformat()
filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': iso_date}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == "Registrar's Order"
assert ledger_filing['data'] == meta_data

### Create Another Annual Report

In [ ]:
filing_name = 'annualReport'
filing_date = business.founding_date + datedelta.datedelta(years=1)
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = ANNUAL_REPORT
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

iso_date = filing_date.date().isoformat()
filing_meta = {
    'legalFilings':['annualReport', 'changeOfAddress', 'changeOfDirectors'],
    filing_name:{
        'annualReportDate': iso_date,
        'annualGeneralMeetingDate': iso_date
}}
meta_data = {**{'applicationDate': iso_date}, **filing_meta}

filing._meta_data = meta_data
filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == f'Annual Report ({date.fromisoformat(iso_date).year})'
assert ledger_filing['data'] == meta_data

### Create a Correction for the last Annual Report

In [ ]:
last_annual_report_filing = filing
filing_name = 'correction'
filing_date = business.founding_date + datedelta.datedelta(years=1, months=1)
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = CORRECTION_AR
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

iso_date = filing_date.date().isoformat()
filing_meta = {'legalFilings':['annualReport', 'changeOfDirectors', 'correction']}
meta_data = {**{'applicationDate': iso_date}, **filing_meta}
filing._meta_data = meta_data
filing.save()

last_annual_report_filing.parent_filing_id = filing.id
last_annual_report_filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == f'Correction - Annual Report ({date.fromisoformat(iso_date).year})'
assert ledger_filing['data'] == meta_data

corrected_ar_filing = next((f for f in ledger if f.get('filingId')==last_annual_report_filing.id), None)

### Create a Special Resolution with Comments

add 5 comments in to the resolution

In [ ]:
filing_name = 'specialResolution'
filing_date = business.founding_date + datedelta.datedelta(years=1, months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = SPECIAL_RESOLUTION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}
filing._meta_data = meta_data

for c in range(5):
    comment = Comment()
    comment.comment = f'this comment {c}'
    filing.comments.append(comment)

filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Special Resolution'
assert ledger_filing['data'] == meta_data
assert ledger_filing['commentsCount'] == c + 1

### Create a Unknown Filing

In [ ]:
filing_name = 'F.18'
filing_date = business.founding_date + datedelta.datedelta(years=1, months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = SPECIAL_RESOLUTION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}
filing._meta_data = meta_data

filing.paper_only = True

filing.save()

ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'F.18'
assert ledger_filing['data'] == meta_data

### Create Avalable on Paper Only Filings

paper only - IncorporationApplication

In [ ]:
filing_name = 'incorporationApplication'
filing_date = business.founding_date + datedelta.datedelta(years=1, months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing'].pop('business')
filing_json['filing']['header']['name'] = filing_name

filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing.paper_only = True

filing._meta_data = meta_data
filing.save()

paper only - Alteration

In [ ]:
filing_name = 'alteration'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name

filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing.paper_only = True

filing._meta_data = meta_data
filing.save()

### Create an Alteration without Court Order

In [ ]:
filing_name = 'alteration'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = ALTERATION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

iso_date = filing_date.date().isoformat()
filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
    'fromLegalType': 'BC',
    'toLegalType': 'BEN'
}}
meta_data = {**{'applicationDate': iso_date}, **filing_meta}

filing.court_order_file_number = 'DEF456'
filing.court_order_date = iso_date
filing.court_order_effect_of_order = 'planOfArrangement'
filing.order_details = 'different order details'

filing._meta_data = meta_data
filing.save()

### Create a Future Effective Alteration

In [ ]:
filing_name = 'alteration'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
FED_date = business.founding_date + datedelta.datedelta(months=filing_month, days=14)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = ALTERATION
filing_json
filing = factory_completed_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
    'fromLegalType': 'BC',
    'toLegalType': 'BEN'
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing.court_order_file_number = ALTERATION['courtOrder']['fileNumber']
filing.court_order_date = ALTERATION['courtOrder']['orderDate']
filing.court_order_effect_of_order = ALTERATION['courtOrder']['effectOfOrder']
filing.order_details = ALTERATION['courtOrder']['orderDetails']

filing.effective_date = FED_date

filing._meta_data = meta_data
filing.save()

### Create a Future Effective Alteration with Court Order

In [ ]:
filing_name = 'alteration'
filing_date = business.founding_date + datedelta.datedelta(months=filing_month)
FED_date = business.founding_date + datedelta.datedelta(months=filing_month, days=14)
filing_month += 1
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = ALTERATION
filing_json
filing = factory_pending_filing(business, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
    'fromLegalType': 'BC',
    'toLegalType': 'BEN'
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing.court_order_file_number = ALTERATION['courtOrder']['fileNumber']
filing.court_order_date = ALTERATION['courtOrder']['orderDate']
filing.court_order_effect_of_order = ALTERATION['courtOrder']['effectOfOrder']
filing.order_details = ALTERATION['courtOrder']['orderDetails']

filing.effective_date = FED_date

filing._meta_data = meta_data
filing.save()

In [ ]:
filing.id

In [ ]:
url = "https://dev.oidc.gov.bc.ca/auth/realms/fcf0kpqr/protocol/openid-connect/token"

payload = "grant_type=client_credentials"
basic_hash = base64.b64encode("entity-notebook-service-account:170e88e8-0a57-4d3c-b7f2-770cc2e1456f".encode()).decode()
basic_hash
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': f'Basic {basic_hash}'
}

response = requests.request("POST", url, headers=headers, data=payload)

token = response.json()['access_token']
# token
assert token

In [ ]:
business.identifier, business.legal_type, business.legal_name

In [ ]:
import json

url = "https://auth-api-dev.apps.silver.devops.gov.bc.ca/api/v1/entities"

payload = {
  "businessIdentifier": business.identifier,
  "corpTypeCode": business.legal_type,
  "name": business.legal_name
}

headers = {
  'Authorization': f'Bearer {token}'
}

r = requests.post(url, headers=headers, json=payload)

r.status_code, r.text

In [ ]:
account = 668  # sev's account
url = f'https://auth-api-dev.apps.silver.devops.gov.bc.ca/api/v1/orgs/{account}/affiliations'

payload = {"businessIdentifier": business.identifier,
           "passCode": ""
          }
headers = {
  'Authorization': f'Bearer {token}'
}

r = requests.post(url, headers=headers, json=payload)

r.status_code, r.text